In [ ]:
# In this lab, the objective is to identify the customers who were active in both May and June, 
# and how did their activity differ between months. To achieve this, follow these steps:

# 1. Establish a connection between Python and the Sakila database.

import mysql.connector
from configparser import ConfigParser
import pandas as pd
from sqlalchemy import create_engine


# Read file config.ini
config = ConfigParser()
config.read('config.ini')

host=config['database']['host']
user=config['database']['user']
password=config['database']['password']
database=config['database']['database']


# Establish the connection
connection = mysql.connector.connect(host,user,password,database)

# Check if the connection was successful
if connection.is_connected():
    print("Connection successful!")

In [ ]:
# 2. Write a Python function called rentals_month that retrieves rental data for a given month 
# and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function 
# should take in three parameters:

# engine: an object representing the database connection engine to be used to establish a connection 
# to the Sakila database.
# month: an integer representing the month for which rental data is to be retrieved.
# year: an integer representing the year for which rental data is to be retrieved.
# The function should execute a SQL query to retrieve the rental data for the specified month and year
# from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [8]:
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/sakila')

def rentals_month(engine, month, year):
    # Formateamos el mes y año para asegurarnos de que están en un rango correcto
    month = str(month).zfill(2)  # Aseguramos que el mes tenga dos dígitos
    year = str(year)
    
    # Definimos el SQL query para seleccionar los datos del mes y año especificado
    query = f"""
    SELECT * FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    
    # Ejecutar el query y leer los datos en un DataFrame
    df = pd.read_sql(query, con=engine)
    
    return df

rentals_df_may = rentals_month(engine,5,2023)

rentals_df_june = rentals_month(engine,6, 2023)

NameError: name 'create_engine' is not defined

In [ ]:
# 3. Develop a Python function called rental_count_month that takes the DataFrame provided by 
# rentals_month as input along with the month and year and returns a new DataFrame containing the 
# number of rentals made by each customer_id during the selected month and year.

#The function should also include the month and year as parameters and use them to name the new 
# column according to the month and year, for example, if the input month is 05 and the year is 
# 2005, the column name should be "rentals_05_2005".

#Hint: Consider making use of pandas groupby()

In [7]:
def rental_count_month(df, month, year):
    # Aseguramos que el formato del mes sea correcto, con dos dígitos
    month = str(month).zfill(2)
    year = str(year)
    
    # Usamos groupby para contar los alquileres por customer_id
    rental_counts = df.groupby('customer_id').size().reset_index(name=f'rentals_{month}_{year}')
    
    return rental_counts


In [ ]:
# 4. Create a Python function called compare_rentals that takes two DataFrames as input containing 
# the number of rentals made by each customer in different months and years. The function should 
# return a combined DataFrame with a new 'difference' column, which is the difference between the 
# number of rentals in the two months.

In [ ]:
def compare_rentals(df1, df2):
    # Supongamos que df1 y df2 tienen una columna 'customer_id' y otra de recuento de alquileres, e.g., 'rentals_05_2023'

    # Fusionamos los dos DataFrames en base a 'customer_id'
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2'))
    
    # Rellenamos NaN con cero porque un cliente puede no tener alquileres en uno de los meses
    combined_df.fillna(0, inplace=True)
    
    # Calculamos la diferencia y creamos una nueva columna 'difference'
    rental_column1 = combined_df.columns[1]
    rental_column2 = combined_df.columns[2]
    combined_df['difference'] = combined_df[rental_column1] - combined_df[rental_column2]
    
    return combined_df

# Ejemplo de uso:
df_may_2023 = rental_count_month(rentals_df_may, 5, 2023)
df_june_2023 = rental_count_month(rentals_df_june, 6, 2023)
comparison_df = compare_rentals(df_may_2023, df_june_2023)